In [ ]:
!pip install transformers accelerate jsonformer bitsandbytes

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, BitsAndBytesConfig
import torch

In [ ]:
print("Loading model and tokenizer...")
model_id = 'meta-llama/Llama-2-7b-chat-hf'
if torch.cuda.is_available():
    config = AutoConfig.from_pretrained(model_id)
    config.pretraining_tp = 1
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        config=config,
        torch_dtype=torch.float16,
        quantization_config=BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    ),
        device_map='auto'
    )
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True, use_cache=True)
print("Loaded model and tokenizer")

Loading model and tokenizer...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded model and tokenizer


In [ ]:
from jsonformer.format import highlight_values
from jsonformer.main import Jsonformer

receipt = {
    "type": "object",
    "properties": {
        "seller": {
            "type": "object",
            "properties": {
                "name": {"type": "string"},
                "address": {"type": "string"},
                "tax_id": {"type": "string"},
            }
        },

        "client": {
            "type": "object",
            "properties": {
                "name": {"type": "string"},
                "address": {"type": "string"},
                "tax_id": {"type": "string"},
            }
        },

        "invoice_number": {
            "type": "number"
        },

        "total":{
            "type": "number"
        },

        "date":{
            "type": "string"
        }
    }
}

In [ ]:
builder = Jsonformer(
    model=model,
    tokenizer=tokenizer,
    json_schema=receipt,
    prompt='''
1
Invoice no: 12060439

Date of issue:
 08/29/2020

Seller:

Brown-Johnson
310 Amanda Corner Suite 472
North William, MN 33119
Tax Id: 981-94-7235
IBAN: GB78IXYE58273701690538

Client:

Calderon-Duran
9884 Roberts Tunnel
North Lindaside, VA 02674
Tax Id: 996-81-8911

ITEMS

No.
 Description
 Qty
 UM
 Net price
 Net worth
 VAT [%]
 Gross
worth

Rectangle Area Rug Wood Grain
Carpet Mat for Living Room
Bedroom - 180x60cm

1.
 4,00
 each
 32,81
 131,24
 10%
 144,36

Rug for living room-Minion And
Banana Area Rugs Living Room
Carpet, Christmas...

2.
 2,00
 each
 29,96
 59,92
 10%
 65,91

Yilong 5'x7.5' Hand Knotted Silk
Area Rug Medium Size Classic
Floor Carpets 0186

3.
 3,00
 each
 2 800,00
 8 400,00
 10%
 9 240,00
SUMMARY

VAT [%]
 Net worth
 VAT
 Gross worth

10%
 8 591,16
 859,12
 9 450,28

Total
 $ 8 591,16
 $ 859,12
 $ 9 450,28. ''',
)

print("Generating...")
output = builder()

highlight_values(output)


Generating...
{
  seller: {
    name: "Brown-Johnson",
    address: "310 Amanda Corner Suite ",
    tax_id: "981-94-723"
  },
  client: {
    name: "Calderon-Duran",
    address: "9884 Roberts Tunnel",
    tax_id: "996-81-891"
  },
  invoice_number: 120604.0,
  total: 85912.0,
  date: "2020-08-29"
}
